In [80]:
#--------- Python Libraries ---------#

import requests
import json
import re
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import csv
import pandas as pd
from datetime import date
import datetime
import dateutil.parser

import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials

import string
import Levenshtein as lev

SPOTIPY_CLIENT_ID = 'a5ad437675474f19bc6c76bb096dee85'
SPOTIPY_CLIENT_SECRET = '69b402da629b41d498354ae80444c3d9'

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET))

from urllib.request import Request, urlopen



In [81]:
#--------- Web crawling for URLs ---------#

main_url = 'http://www.undertoner.dk/kategori/koncertanmeldelser/'
req = Request(main_url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, 'html.parser')


#print(source)
print(soup)

<!DOCTYPE html>

<!--[if IE 8]><html class="ie8"><![endif]-->
<!--[if IE 9]><html class="ie9"><![endif]-->
<!--[if gt IE 8]><!--> <html lang="da-DK"> <!--<![endif]-->
<head>
<script async="" src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
<script>
  (adsbygoogle = window.adsbygoogle || []).push({
    google_ad_client: "ca-pub-5943743144286183",
    enable_page_level_ads: true
  });
</script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="user-scalable=yes, width=device-width, initial-scale=1.0, maximum-scale=1, minimum-scale=1" name="viewport"/>
<!--[if lt IE 9]>
	<script src="http://www.undertoner.dk/wp-content/themes/voice/js/html5.js"></script>
<![endif]-->
<title>Koncerter Arkiv - Undertoner</title>
<!-- This site is optimized with the Yoast SEO plugin v12.0 - https://yoast.com/wordpress/plugins/seo/ -->
<meta content="Find de nyeste koncertanmeldelser på onlinemagasinet Undertoner. Leveret til dig af musikjournalister."

In [82]:
list_of_URLs = []

# Loop through all review links to find concert reviews specifically

#links = soup.find_all('h2')
#print(links)

for link in soup.find_all('h2',{'class' : 'entry-title'}):
    link = link.find('a').get('href')
    list_of_URLs.append(link)
    print('Found concert review URL')


# Remove duplicate URLs in list
list_of_URLs = list(set(list_of_URLs))
print(list_of_URLs)

Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
['http://www.undertoner.dk/2020/08/yune-27-08-20-empire-bio-kbenhavn/', 'http://www.undertoner.dk/2020/09/frigjort-festival-mag-kogekunst-05-09-20-loppen-christiania/', 'http://www.undertoner.dk/2020/02/the-comet-is-coming-22-02-20-journey-fest-koebenhavn/', 'http://www.undertoner.dk/2020/07/nicklas-soerensen-johannes-lund-henrik-pultz-melbye-24-07-20-alice-koebenhavn/', 'http://www.undertoner.dk/2020/09/uhoert-festival-leizure-mari-natlyst-04-09-20-koebenhavn/', 'http://www.undertoner.dk/2020/02/pottery-23-02-20-ideal-bar-koebenhavn/', 'http://www.undertoner.dk/2020/02/moses-boyd-24-02-20-hotel-cecil-koebenhavn/', 'http://www.undertoner.dk/2

In [84]:
# 2 test URLs
list_of_URLs = ['http://www.undertoner.dk/2020/08/yune-27-08-20-empire-bio-kbenhavn/']

# Create empty dictionary
critic_review_dict = {} 
critic_review_dict['concerts'] = []

# Loop - For every link in the URL_links do the following with the reviews:
for i in list_of_URLs:

    # Reset values
    critic_name = 'Undertoner' 
    author = ''
    url = ''
    headline = ''
    text = ''
    rating = 0
    max_rating = 6
    tribute_rating = 0
    artist_name = ''
    venue_name = ''
    venue_place = ''
    publication_date = ''
    concert_date = ''
    lang = 'da'
    genre = []
    venue_is_online = False
    
    url = i
    
    # Get the source code of the webpage as text
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')

    # Find headline
    try:
        headline = soup.h1.text
        print(headline)
        print('Headline found')
    
    except:
        print('No headline found')
        
    # Split headline
    try:
        split_headline = headline.split(',')
        print(split_headline)
    except:
        print('Could not split headline')
        
    
    # Find text
    text_list = []
    
    try:
        intro_text = soup.find('div',{'class' : 'entry-headline'})
        intro_text = intro_text.find_all('p')
        intro_text = [re.sub(r'<.+?>',r'',str(a)) for a in intro_text]
        for i_text in intro_text:
            text_list.append(i_text)
        print('Intro text intro found')
        print(text_list)
    
    except:
        print('No intro text found')
    
    try:
        main_text = soup.find('div',{'class' : 'entry-content'})
        main_text = main_text.find_all('p')
        main_text = [re.sub(r'<.+?>',r'',str(a)) for a in main_text]
        
        for i_text in main_text:
            text_list.append(i_text)
        
        # Remove photographer info
        for i_text in text_list:
            if 'fotograf:' in i_text.lower():
                text_list.remove(i_text)
                
        text = '\n\n'.join(text_list)
        print('Text found')
        
    except:
        print('No text found')
    
    # Find artist name
    try:
        artist_name = split_headline[0]
        print('artist name found')
    except:
        artist_name = ''
        print('No artist name found')
        continue
    
    # Find venue name
    try:
        venue_name = split_headline[-2]
        print('Cenue name found')
        
    except:
        print('No venue name found')
    
    # Find venue place
    try:
        venue_place = split_headline[-1]
        print('Venue place found')
    except:
        print('No venue place name found')
    
    
    # Find publication date
    try:    
        publication_date = soup.find('meta',{'property' : 'article:published_time'})
        publication_date = publication_date["content"]
        publication_date = dateutil.parser.parse(publication_date)
        print('publication date found')

    except:
        publication_date = ''
        print('No publication_date found') 
        continue
    
    # Find concert date
    try:
        concert_date = split_headline[1]
        
        # Format date
        concert_date = concert_date.lstrip()
        default_time = 'T00:00:00+01:00'
        concert_date = concert_date+default_time
        concert_date = dateutil.parser.parse(concert_date, dayfirst=True)
        print('Concert Date found')
    except:
        print('No concert date found')
    
    # Find Author
    try:
        author = soup.find('span',{'class' : 'vcard author'})
        author = author.find('a').text
        print('Author found')
        
    except:
        print('No author found')
    
    # Find rating
    try:
        rating = soup.find('div',{'id':'grade'})
        rating = len(rating.find_all('img',{'alt': '★'}))
        tribute_rating = (5/max_rating)*rating
        tribute_rating = round(tribute_rating, 2)
        print('Rating found')
    except:
        print('No rating found')
    
    ########## Spotify lookup ############
    try:
        # Preprocessing: Remove all within () and punctuation, set lowercase
        preprocessed_name = re.sub("([\(\[]).*?([\)\]])", "\g<1>\g<2>", artist_name)
        preprocessed_name = preprocessed_name.lower()
        preprocessed_name = preprocessed_name.translate(str.maketrans('', '', string.punctuation))
        print('Name preprocessing:', artist_name, '=' ,preprocessed_name)

        # Reset spotify values
        spotify_match = False
        artist_name_spotify = ""
        artist_img_spotify = ""
        artist_spotify_id = ""
        subgenres_spotify = []
        artist_popularity_spotify = ""

        # Make Spotify lookup on raw artist name
        results = spotify.search(q='artist:' + artist_name, type='artist')
        items = results['artists']['items']
        
        # If spotify returns no results on raw artist name search, make Spotify lookup on preprocessed artist name

        if len(items) == 0:
            print('No result on raw name, trying preprocessed name...')
            results = spotify.search(q='artist:' + preprocessed_name, type='artist')
            items = results['artists']['items']

        # If spotify returns results on raw artist name
        if len(items) > 0:

            print('found', len(items), 'results')
            print('searching for exact string match...')

            # First check all results for exact string match
            for ii in range(len(items)):
                artist = items[ii]

                # Set lower case and match songkick name with spotify name
                if artist_name.lower() == artist['name'].lower():
                    print(artist)

                    spotify_match = True
                    print('Found exact string match on index', ii, artist['name'])
                    
                    artist_name_spotify = artist['name']
                    artist_img_spotify = artist['images'][0]['url']
                    artist_spotify_id = artist['id']
                    subgenres_spotify = artist['genres']
                    artist_popularity_spotify = artist['popularity']

                    break

                else:
                    print('No exact match on result from index ', ii, artist['name'])

            # If no match, then check all spotify results for fuzzy string match above 90%
            if spotify_match == False:
                print('searching for fuzzy string match...')

                for iii in range(len(items)):
                    artist = items[iii]

                    Distance = lev.distance(preprocessed_name,artist['name'].lower()),
                    Ratio = lev.ratio(preprocessed_name,artist['name'].lower())

                    if Ratio > 0.90:

                        spotify_match = True
                        print('Found fuzzy string match on index', iii, artist['name'])
                        print('with ratio:', Ratio)

                        artist_name_spotify = artist['name']
                        artist_img_spotify = artist['images'][0]['url']
                        artist_spotify_id = artist['id']
                        subgenres_spotify = artist['genres']
                        artist_popularity_spotify = artist['popularity']
                        break

                    else:
                        print('No fuzzy match on result from index ', iii, artist['name'])

        else:
            print('no Spotify search results at all from the preprocessed string')
    except:
        print('unsuccesful spotify lookup')

    print('Critic:', critic_name)
    print('Author:', author)
    print('URL:', url)
    print('Artist:', artist_name)
    print('Venue:', venue_name)
    print('Venue Place:', venue_place)
    print('Rating:', str(rating))
    print('Max Rating:', str(max_rating))
    print('Tribute Rating:', str(tribute_rating))
    print('Publication Date:', publication_date)
    print('Concert Date:', concert_date)
    print('artist spotify name:', artist_name_spotify)
    print('artist spotify image:', artist_img_spotify)
    print('artist spotify ID:', artist_spotify_id)
    print('artist spotify subgenres:', subgenres_spotify)
    print('artist spotify popularity:', artist_popularity_spotify)
    print('Language of review:', lang)
    print('Headline:', headline)
    print('Text:', text)
    
    # Set dictionary values

    temp_dict = {}
    temp_dict['critic_name'] = critic_name
    temp_dict['author'] = author
    temp_dict['url'] = url
    temp_dict['artist_name'] = artist_name
    temp_dict['artist_name_spotify'] = artist_name_spotify
    temp_dict['artist_img_spotify'] = artist_img_spotify
    temp_dict['artist_spotify_id'] = artist_spotify_id
    temp_dict['artist_popularity_spotify'] = artist_popularity_spotify
    temp_dict['genre'] = genre
    temp_dict['subgenre_spotify'] = subgenres_spotify
    temp_dict['publication_date'] = publication_date
    temp_dict['concert_date'] = concert_date
    temp_dict['venue_name'] = venue_name
    temp_dict['venue_place'] = venue_place
    temp_dict['rating'] = rating
    temp_dict['tribute_rating'] = tribute_rating
    temp_dict['max_rating'] = max_rating
    temp_dict['review_language'] = lang
    temp_dict['headline'] = headline
    temp_dict['text'] = text

    critic_review_dict['concerts'].append(temp_dict)

    
    
    


Yune, 27.08.20, Empire Bio, København
Headline found
['Yune', ' 27.08.20', ' Empire Bio', ' København']
Intro text intro found
['Yune spillede forsinket releasekoncert i Empire Bio, men det velspillende aarhusianske bands ørkenrock fik desværre aldrig rigtig sandet til at fyge, så det gjorde indtryk.']
Text found
artist name found
Cenue name found
Venue place found
publication date found
Concert Date found
Author found
Rating found
Name preprocessing: Yune = yune
found 10 results
searching for exact string match...
No exact match on result from index  0 Yuneer Gainz
No exact match on result from index  1 Yunel
No exact match on result from index  2 Yunell
No exact match on result from index  3 yunè pinku
No exact match on result from index  4 Yuneak
{'external_urls': {'spotify': 'https://open.spotify.com/artist/2EjyhrxUN4jMed15kpV9i8'}, 'followers': {'href': None, 'total': 1}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/2EjyhrxUN4jMed15kpV9i8', 'id': '2EjyhrxUN4jMed15kpV9